<a href="https://colab.research.google.com/github/RewGuardiano/AI_Module_Assignment/blob/main/RewGuardiano_Genetic_Algorithm_Assignment_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Genetic Algorithm Assignment
30% of the overall grade for this module

Marks indciated in sections below are based on percentage of marks allocated for this module

In this assignment you must choose a problem, and attempt to use the Genetic Alogrithm that we developed in class to solve this problem.





## The Problem         **(~30%)**

*   Description of the problem





---
# ** Sudoku Solver**

 This will generate valid Soduku solutions.

---
- Sudoku is a logical-based number placement puzzle.
- It consists of 9x9 grid which is divided into 9 3x3 sub-grids like boxes.
- The digits 1 through 9 should appear exactly once in each row.
- The digits 1 through 9 should appear exactly once in each column.
- Every 3x3 sub-grid has exactly one instance of each of the integers 1 through 9.

**Completing the grid while keeping to the given constraints is the task, and certain numbers are pre-filled as clues.**


---



*   Discussion of the suitablity of Genetic Algorithms


---
# The Genetic Algorithm is ideal for solving Sudoku for following reasons:

**Managing Constraints:**
By creating a fitness function that provides adherence to rules, the genetic algorithm can be adjusted towards correct solutions.

**Flexibility in Representation:** The grid,rows,columns and sub-grids make a clear represetation as a indiviual/chromosome in the GA. Applying genetic operators like crossover and mutation is hence straightforward.

**Global Search Capability:** Since solving Sudoku requires satisfying several constraints (rows, columns, and sub-grids), GAs' ability to do a global search is significant.

**As seen with the Cost Function**
 - Penalises when Soduku rules are violated e.g. duplicated numbers in rows and columns.
 - The perfect cost will be equal to 0. The cost will increase with the amount of constraints each individual violates.

**Exploration through Population:** Unlike traditional backtracking techniques that investigate solutions one after the other, GAs investigate several possibilities simultaneously (through the population), increasing the possibility of discovering a workable solution.


**The Benefit of Mutation and Crossover:** By expanding the pool of potential solutions, processes like mutation and crossover keep the algorithm from being trapped in one singe optimal solution.

---



In [ ]:
import numpy as np
import random
import matplotlib.pyplot as plt

def generate_indiviual():
  sudoku = np.zeros((9,9), dtype=int)  # Initialize empty grid
  for i in range(9):
    numbers = list(range(1,10))
    random.shuffle(numbers)
    sudoku[i]= numbers
  return sudoku

**Firstly, this function generates a random soduku grid. The code: **


*   Creates a 9x9 grid filled with zeros
*   for each row it creates a random numbers from 1-9
*   For optimisation, It ensures that every row already meets the Sudoku requirement of having direct numbers of 1-9. Since the cost function will check duplicates from columns and subgrids.  
*   Then returns the entire grid as a NumPy array







*   Complexity of the problem  (Overall marks allocated based on ..)

# The problem and the cost function   **(~20%)**

The cost function will:

*   Look for duplicates in the columns.
*   Verify indiviual 3x3 grids from the whole Soduku grid.
*   In both columns and each 3x3 grids, the more duplicates or violations results in higher cost function.

*  Sum of all violations determines how good the cost function of a solution to a soduku board is. The cost of a perfect Soduku solution should be zero.








In [ ]:
def visualize_sudoku(sudoku):
    fig, ax = plt.subplots(figsize=(6, 6))
    ax.set_aspect("equal")

    # Draw minor grid lines (thin cell borders)
    for i in range(10):
        lw = 1 if i % 3 != 0 else 3  # Thicker lines for 3x3 sub-grid
        ax.plot([i, i], [0, 9], color="black", linewidth=lw)  # Draws Vertical lines
        ax.plot([0, 9], [i, i], color="black", linewidth=lw)  # Draws Horizontal lines

    # Hide axis ticks
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_xticklabels([])
    ax.set_yticklabels([])

    # Fill in the numbers
    for i in range(9):
        for j in range(9):
            num = sudoku[i, j]
            if num != 0:
                ax.text(j + 0.5, 8.5 - i, str(num), ha='center', va='center',
                        fontsize=16, fontweight='bold')

    plt.xlim(0, 9)
    plt.ylim(0, 9)
    plt.show()

def check_cost(sudoku):
  #lists
    column_duplicates = []
    subgrid_duplicates = []

    # Check columns
    column_cost = 0
    for col in range(9):
      #dictionary to keep track of numbers already encountered
        seen = {}
        for row in range(9):
            num = sudoku[row, col]
            if num in seen:
                column_duplicates.append((row, col))
                column_cost += 1
            else:
                seen[num] = (row, col) #else stores its indices coordinates if not a duplicate

    # Check 3x3 subgrids
    subgrid_cost = 0
    for box_row in range(3):
        for box_col in range(3):
            seen = {}
            #inner for loops iterate through the cells inside the subgrids
            for i in range(3):
                for j in range(3):
                    row = box_row * 3 + i
                    col = box_col * 3 + j
                    num = sudoku[row, col]
                    if num in seen:
                        subgrid_duplicates.append((row, col))
                        subgrid_cost += 1
                    else:
                        seen[num] = (row, col)

    total_cost = column_cost + subgrid_cost
    return total_cost, column_duplicates, subgrid_duplicates

# Generate a random Sudoku grid
sudoku_grid = generate_indiviual()

# Visualize the Sudoku grid
visualize_sudoku(sudoku_grid)

#calculating cost
cost, column_duplicates, subgrid_duplicates = check_cost(sudoku_grid)
# Print results
print("Total cost:", cost)
if cost == 0:
    print("The Sudoku grid is perfect!")
else:
    print("Column duplicate indices:", column_duplicates)
    print("Subgrid duplicate indices:", subgrid_duplicates)

**The cost function assesses a Sudoku grid's adheres with the rules**

The code above:


1.   **Check constraints in the columns in the Grid**

*   Iterates through each of the 9 columns in
*   Uses a dictonary (seen variable) to keep track of the numbers seen

*   When a duplicate is discovered in the same column, the cost is increasesd in the column_cost variable.
*   The column_duplicates variable then keeps tracks of duplicates indices.
*   The column cost variable increases by one if a duplicate is found




2. **Checking constraints of the subgrids in the grid**


     for box_row in range(3):
        for box_col in range(3):
            seen = {}
            #inner for loops iterate through the cells inside the subgrids
            for i in range(3):
                for j in range(3):
*   Iterates through each of the nine 3x3 subgrids
*   Nested for loop goes over each indiviual subgrids rows and columns to check for duplicates.
*   subgrid_cost variable increments with every duplicate.
* Indices of duplicates are also recorded in a subgrid_duplicates variable.

3. **Total Cost Calculation**

* Addition of column_cost and subgrid_cost to get the total_cost
* A complete soduku grid is indicated if the costs function is zero.
*  Total cost is printed out showing all the duplicates.

















# The Individual **(~30%)**


*   Chromosone
*   Crossover
*   Mutation



## Discussion and justification on the approaches taken for the above

---


**Individual**
 - Each individual represents one possible Sudoku solution.
 - The solution is encoded in a chromosome, often a 9x9 grid.

**Chromosome**
- an individual's chromosome is represented as a 9x9 NumPy array, with a number between 1 and 9 in each cell
- This chromosome is initialized with the generate_individual() method, which fills each row with a random combination of 1 through 9:

**Crossover & Mutation**
- Crossover: Combines two parent solutions to produce new offspring.

 - The crossover uses the partial mapped crossover logic, modified to work with subsets of rows.
 - It uses a mapping to maintain row uniqueness, the supplied crossover function chooses two to four random rows and, within each, switches a portion of cells between parents. The implementation is as follows:
 - **Using a mapping dictionary**, this PMX-inspired crossover resolves duplicates outside of the swapped segment after switching segments (such as columns 2–5) between parents row-wise. For instance, offspring1 may initially take [2, 1, 8] from parent2 if parent1's row 3 is [9, 3, 2, 1, 8, 4, 5, 6, 7] and parent2's is [6, 1, 7, 8, 2, 9, 5, 3, 4], with crossover points 2 and 4. To ensure that the row stays a permutation of 1–9, the mapping (e.g., 2→7, 1→8, 8→2) then modifies the remaining locations to remove duplicates.

- Mutation: Mutation introduces random variations to maintain population diversity and ensures validity.
  - In the Sudoku genetic algorithm, mutation swaps two numbers within the same row of the grid, preserving the row’s uniqueness.
  - For example, switching positions 1 and 3 (5 and 7) in a row [4, 5, 2, 7, 3, 9, 1, 6, 8] yields [4, 7, 2, 5, 3, 9, 1, 6, 8]. While changing the column and subgrid configurations, which the cost function assesses, this maintains the row's validity.


---



In [1]:
def crossover_function(parent1,parent2):

    offspring1 = parent1.copy()
    offspring2 = parent2.copy()

    # Select a subset of rows to crossover
    num_rows = random.randint(2, 4)
    crossover_rows = random.sample(range(9), num_rows)

    for row in crossover_rows:
        # Generate two crossover points
        cx_point1, cx_point2 = sorted(random.sample(range(9), 2))

        # Create mapping between parents for this segment
        mapping = {}
        for i in range(cx_point1, cx_point2 + 1):
            mapping[parent1[row, i]] = parent2[row, i]
            mapping[parent2[row, i]] = parent1[row, i]

        # Apply segment from second parent to first offspring
        offspring1[row, cx_point1:cx_point2+1] = parent2[row, cx_point1:cx_point2+1]

        # Apply segment from first parent to second offspring
        offspring2[row, cx_point1:cx_point2+1] = parent1[row, cx_point1:cx_point2+1]

        # Fix the remaining elements in the row to ensure 1-9 appear exactly once
        for i in range(9):
            if i < cx_point1 or i > cx_point2:

                while offspring1[row, i] in offspring1[row, cx_point1:cx_point2+1]:
                    offspring1[row, i] = mapping.get(offspring1[row, i], offspring1[row, i])


                while offspring2[row, i] in offspring2[row, cx_point1:cx_point2+1]:
                    offspring2[row, i] = mapping.get(offspring2[row, i], offspring2[row, i])

    # Randomly select one of the two offspring to return
    return offspring1 if random.random() < 0.5 else offspring2

In [2]:
def mutation_function(sudoku, fixed_positions, mutation_rate=0.1):

    mutated_sudoku = sudoku.copy()

    for row in range(9):
        if random.random() < mutation_rate:

            swap_index1, swap_index2 = random.sample([i for i in range(9) if not fixed_positions[row, i]], 2)

            mutated_sudoku[row, swap_index1], mutated_sudoku[row, swap_index2] = \
            mutated_sudoku[row, swap_index2], mutated_sudoku[row, swap_index1]

    return mutated_sudoku


## Running the algorithm  **(~10%)**

*   Parameter choices
*   Modifications (if any) to run_genetic
*   Rationale for the above



---

(Double click then ...)  Type Here...

---



In [ ]:
#  Parameter class here

In [ ]:
# Run Genetic method here

In [ ]:
#  Running of the algorithm with outputs here

In [ ]:
#  If changes to params or reruns of iterations dont overwrite, create more cells and copy code down to show evolution of final solution

## Results and conclusions    **(~10%)**

---

(Double click then ...)  Type Here...

---

